# 生成 annotation.csv & coords.pickle

In [1]:
import os
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import csv
import math
import pickle
from glob import glob
from tqdm import tqdm
from utils import *

lidc_folder = "/home/cbel/Desktop/zhe/LIDC"
data_folder = "/home/cbel/Desktop/zhe/data"

In [2]:
check_folder(data_folder)

/home/cbel/Desktop/zhe/data3建立成功


In [3]:
# 擷取 meta.csv 中為 CT 的 uid

df_meta = pd.read_csv(lidc_folder + "/LIDC-IDRI_MetaData.csv")  # length = 1398

df_CT = df_meta[df_meta["Modality"] == "CT"]  # length = 1018
df_CT = df_CT.sort_values(by = ["Patient Id"])

array_CT_seriesuid = df_meta[df_meta["Modality"] == "CT"]["Series UID"].to_numpy()

tmp = []
id_uid = {}  # 1010 patient id 對應 1018 series uid
for i in range(1018):
    if df_CT.iloc[i]["Patient Id"] not in tmp:
        tmp.append(df_CT.iloc[i]["Patient Id"])
        id_uid[df_CT.iloc[i]["Patient Id"]] = [df_CT.iloc[i]["Series UID"]]
    else:
        id_uid[df_CT.iloc[i]["Patient Id"]].append(df_CT.iloc[i]["Series UID"])

In [4]:
# 擷取所需要的 xml file

xml_folder = lidc_folder + "/tcia-lidc-xml"

xml_path = []
for file in os.listdir(xml_folder):
    if file[-3:] == "xml":
        xml_path.append(xml_folder + "/" + file)
    else:
        path = glob(xml_folder + "/" + file + "/*.xml")
        for p in path:
            xml_path.append(p)

seriesuid_xmlpath = {}

tmp_uid = []
for i in range(len(xml_path)):
    tree = ET.parse(xml_path[i])
    root = tree.getroot()
    
    for ResponseHeader in root.findall("{http://www.nih.gov}ResponseHeader"):
        seriesuid = ResponseHeader.find("{http://www.nih.gov}SeriesInstanceUid").text
        studyuid = ResponseHeader.find("{http://www.nih.gov}StudyInstanceUID").text
        if seriesuid in array_CT_seriesuid and seriesuid not in tmp_uid:
            tmp_uid.append(seriesuid)
            seriesuid_xmlpath[seriesuid] = xml_path[i]

In [5]:
# 產生 1018 個 case 對應的 dicom path 及 series uid
# 目錄 : dicom folder --> folders(study uid) --> folders(seried uid) --> dicom data

n = 0
case_dcmpath = {}  # case - data path, 1018 個
case_serieduid = {}  # case - series uid, 1018 個
dcmfolder2_case = {}  # 下方 list3.2.csv 的 data 轉換會用到

for i in tqdm(range(1, 1013)):
    patient_id = "LIDC-IDRI-%04d" %i
    dcm_folder1 = glob(lidc_folder + "/dicom/" + patient_id + "/*")  # 只有 1010 個資料夾
    if len(dcm_folder1) != 0:
        for j in range(len(dcm_folder1)):
            dcm_folder2 = os.listdir(dcm_folder1[j])
            for k in range(len(dcm_folder2)):
                file_seriesuid_5 = dcm_folder2[k][-5:]  # 1 個 file 對應 1 個 series uid(末 5 碼)
                seriesuids = id_uid[patient_id]  # 1 個 patient 可能有 1~2 個 serieduid(case)
                seriesuids_5 = list(map(lambda x:x[-5:], id_uid[patient_id]))  # 末 5 碼
                if file_seriesuid_5 in seriesuids_5:
                    n+=1
                    case = "%04d" %n
                    case_dcmpath[case] = dcm_folder1[j] + "/" + dcm_folder2[k]
                    if "." in dcm_folder2[k]:
                        dcm_folder2_name = dcm_folder2[k].split(".")[0]
                        dcmfolder2_case[dcm_folder2_name + patient_id] = case
                    else:
                        dcmfolder2_case["0" + patient_id] = case
                    for x in range(len(seriesuids)):
                        if seriesuids[x][-5:] == file_seriesuid_5:
                            case_serieduid[case] = seriesuids[x]

100%|██████████| 1012/1012 [00:00<00:00, 39420.44it/s]


In [6]:
# 將 list3.2.csv 的 case 從 1~1012 轉換為 1~1018

df_list = pd.read_csv(lidc_folder + "/list3.2.csv")
dflist_casenumber = len(np.unique(df_list["case"]))
new_list = df_list
new_cases = []

for i in range(len(df_list)):
    old_case = df_list["case"][i]
    patient_id = "LIDC-IDRI-%04d" %old_case
    filename = str(df_list["scan"][i])
    new_case = int(dcmfolder2_case[filename + patient_id])  # 找出對應的編號
    new_cases.append(new_case)
    new_list.loc[i, "case"] = new_case
new_list.to_csv(lidc_folder + "/new_list.csv", index=0)
    
array_newlist = new_list["case"].to_numpy()
uni_newlist = np.unique(array_newlist)
print("%d patients to %d cases" %(dflist_casenumber, len(uni_newlist)))

868 patients to 876 cases


In [7]:
# parse xml

case_id_ma = {}  # 883 cases
case_id_uid_coord = {}  # 883 cases
case_id_uid_anchor = {}  # 883 cases
uid_z = {}
a = []

for i in tqdm(range(1018)):
    case = "%04d" %(i+1)
#     print(case)
    xml_path = seriesuid_xmlpath[case_serieduid[case]]
#     print(xml_path)

    tree = ET.parse(xml_path)
    root = tree.getroot()

    id_ma = {} # nodule id - malignancy
    id_uid_coord = {} # nodule id - sop uid
    id_uid_anchor = {} # nodule id - sop uid
    for readingSession in root.findall("{http://www.nih.gov}readingSession"): # doctor

        for unblindedReadNodule in readingSession.findall("{http://www.nih.gov}unblindedReadNodule"): # nodules

            noduleID = unblindedReadNodule.find("{http://www.nih.gov}noduleID").text
            if noduleID[0] == "0" and len(noduleID) > 1:
                    noduleID = cut_0(noduleID)
#             print("noduleID:", noduleID)

            uid_coord = {}
            uid_anchor = {}
            for characteristics in unblindedReadNodule.findall("{http://www.nih.gov}characteristics"): # malignancy
                if characteristics.find("{http://www.nih.gov}malignancy") != None:
                    malignancy = int(characteristics.find("{http://www.nih.gov}malignancy").text)                   
                    id_ma[noduleID] = malignancy
#                     print("malignancy:", malignancy)

                    for roi in unblindedReadNodule.findall("{http://www.nih.gov}roi"): # roi
                        z = roi.find("{http://www.nih.gov}imageZposition").text
                        sop_uid = roi.find("{http://www.nih.gov}imageSOP_UID").text

                        x_coords = []
                        y_coords = []
                        xy_coords = []
                        for edgeMap in roi.findall("{http://www.nih.gov}edgeMap"): # x, y
                            x = int(edgeMap.find("{http://www.nih.gov}xCoord").text)
                            y =  int(edgeMap.find("{http://www.nih.gov}yCoord").text)
                            x_coords.append(x)
                            y_coords.append(y)
                            xy_coords.append([x, y])
                        x_max = get_max_coord(x_coords)
                        y_max = get_max_coord(y_coords)
                        x_min = get_min_coord(x_coords)
                        y_min = get_min_coord(y_coords)

                        uid_coord[sop_uid] = xy_coords
                        uid_anchor[sop_uid] = [x_max+1, y_max+1, x_min-1, y_min-1]
                        uid_z[sop_uid] = z
            if uid_coord or uid_anchor != {}:
                id_uid_coord[noduleID] = uid_coord
                id_uid_anchor[noduleID] = uid_anchor

    #     print("-------------doctor-------------")
    if id_ma or id_uid_anchor or id_uid_coord != {}:
        a.append(int(case))
        case_id_ma[case] = id_ma
        case_id_uid_coord[case] = id_uid_coord
        case_id_uid_anchor[case] = id_uid_anchor

100%|██████████| 1018/1018 [00:09<00:00, 109.04it/s]


In [8]:
# 產生 classification annotation.csv (3 mm <= nodule <= 30 mm)

cla_file = open(data_folder + "/annotation.csv", "w")
writer = csv.writer(cla_file)
writer.writerow(["case", "nodule_number_percase", "slice_number_pernodule", "total_nodule_number" ,"total_slice_number", 
                 "radiologists", "SOP_UID", "malignancy", "z", "xmax", "ymax", "xmin", "ymin", "diam"])

case_noduleids = {}
case_number_sopuid_coords = {}

total_slice_number = 0
total_nodule_number = 0

for i in tqdm(range(len(uni_newlist))):
    case = "%04d" %uni_newlist[i]
    case_unit = new_list[new_list["case"] == int(case)]
    nodule_number_percase = 0
    case_noduleids[case] = []
    number_sopuid_coords = {}
    for j in range(len(case_unit)): # traverse 每個 case 的 nodule 數量
        nodule_same = []
        ma_subset = []
        diam = case_unit.iloc[j]["eq. diam."]
        for k in range(9, 13): # 查看有幾個醫生發現
            noduleid = str(case_unit.iloc[j][k])
            if noduleid != "nan" and noduleid in case_id_ma[case].keys():
                malignancy = case_id_ma[case][noduleid]
                nodule_same.append(noduleid)
                ma_subset.append(malignancy)
        if len(ma_subset) >= 3: # 擷取至少 x 位以上醫生發現的 nodule
            nodule_number_percase += 1
            total_nodule_number += 1
            # get the median of the likelihoods of malignancy
            cal_ma = np.median(ma_subset)
            if cal_ma > 3:
                malignancy = 1
            elif cal_ma < 3:
                malignancy = 0
            else:
                malignancy = 2
            sopuid_coords = {}
            rad_number = len(nodule_same)
            case_noduleids[case].append(nodule_same)
            all_uids = []  # 長度為有標註的醫生數目，裡面存放不同醫生標註的 sopuids，一樣的 sopuid 為同一張 slice
            for noduleid in nodule_same:
                for uid in case_id_uid_anchor[case][noduleid].keys():
                    all_uids.append(uid)
            # 對 1 位以上醫生標註的 sopuids 取交集
            in_uids = []
            uniques, counuts = np.unique(all_uids, return_counts=True)
            for c in range(len(counuts)):
                if counuts[c] >= 1:
                    in_uids.append(uniques[c])
            slice_number_pernodule = 0
            for in_uid in in_uids:
                slice_number_pernodule += 1
                total_slice_number += 1
                all_anchor = []
                all_coords = []
                for noduleid in nodule_same:
                    for uid in case_id_uid_anchor[case][noduleid].keys():
                        if uid == in_uid:
                            anchor = case_id_uid_anchor[case][noduleid][uid]
                            coords = case_id_uid_coord[case][noduleid][uid]
                            all_anchor.append(anchor)
                            all_coords.append(coords)
                xmax, ymax, xmin, ymin, = list(map(lambda x:np.ceil(x), np.mean(all_anchor, axis=0)))  # 取平均的 max, min
                z = uid_z[in_uid]
                sopuid_coords[in_uid] = all_coords
                writer.writerow([case, nodule_number_percase, slice_number_pernodule, total_nodule_number, total_slice_number,
                                 rad_number, in_uid, malignancy, z, xmax, ymax, xmin, ymin, diam])  
            if sopuid_coords != {}:
                number_sopuid_coords[nodule_number_percase] = sopuid_coords
    if number_sopuid_coords != {}:
        case_number_sopuid_coords[case] = number_sopuid_coords
cla_file.close()

cla_pickle = open(data_folder + "/coords.pickle", "wb")
pickle.dump(case_number_sopuid_coords, cla_pickle)
cla_pickle.close()

100%|██████████| 876/876 [00:02<00:00, 390.61it/s]
